In [1]:
import pandas as pd

# NFL Big Data Bowl Clustering Extra Point

## Import Data

In [2]:
plays = pd.read_csv("../../NFL Data/Data/plays.csv")
games = pd.read_csv("../../NFL Data/Data/games.csv")
players = pd.read_csv("../../NFL Data/Data/players.csv")
track_ep2018 = pd.read_csv("../../NFL Data/Data/tracking_ep18.csv")
track_ep2019 = pd.read_csv("../../NFL Data/Data/tracking_ep19.csv")
track_ep2020 = pd.read_csv("../../NFL Data/Data/tracking_ep20.csv")

In [3]:
tracking_fep = pd.read_csv("../../NFL Data/Data/tracking_fep.csv")

## Preprocessing Players Data

In [4]:
from Preprocessing_makers import preprocess_players

In [5]:
players_df = preprocess_players(players)

## Preprocessing Tracking Data

In [6]:
from Preprocessing_makers import preprocess_tracking, preprocess_football_track


In [7]:
ep_tracking_2018, ep_tracking_2019, ep_tracking_2020 = preprocess_tracking(track_ep2018, track_ep2019, track_ep2020, plays, 'Extra Point')


In [8]:
ep_tracking_ball = preprocess_football_track(ep_tracking_2018, ep_tracking_2019, ep_tracking_2020)


## Preprocessing Play Data

In [9]:
from Preprocessing_makers import preprocess_play
from Cleaning_data import drop_by_index_difference

In [10]:
play_df = preprocess_play(plays)

In [11]:
play_df = drop_by_index_difference(play_df, ep_tracking_ball, 'extra_point_attempt')

## Make Extra Point Data Frame

In [12]:
from Preprocessing_makers import make_extraPoint

In [13]:
extra_point_df = make_extraPoint(play_df, players_df)

## Freature Engineering

In [14]:
from Feature_engineering import endzone_y_pos, expected_endzone_y_pos, kicker_core_dist

### Endzone y-Position

In [15]:
extra_point_df = endzone_y_pos(extra_point_df, ep_tracking_ball)

### Expected Trajectory Position

#### I don't know what I am doing past this point and too tired to figure it out tonight. 

In [16]:
extra_point_df = expected_endzone_y_pos(extra_point_df, ep_tracking_ball, 'extra_point_attempt')

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)


In [17]:
extra_point_kick_df = extra_point_df[extra_point_df['specialTeamsResult'].isin(['Kick Attempt Good', 'Kick Attempt No Good'])]

In [18]:
extra_point_kick_df = expected_endzone_y_pos(extra_point_kick_df, ep_tracking_ball, 'extra_point_attempt')

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:188: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2-y1)/(x2-x1)
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_play['expected_endzone_y'] = pt_play.index.map(


### Kicker Core Distance

In [19]:
extra_point_kick_df = kicker_core_dist(extra_point_kick_df, ep_tracking_2018, ep_tracking_2019, ep_tracking_2020, ep_tracking_ball, 'extra_point_attempt', k=3)


/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

...

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL Data/Plays/Feature_engineering.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opposing_team_players['kicker_dist'] = l2_norm(kicker_x, kicker_y, opposing_team_players['x'], opposing_team_players['y'])
/Users/marcobertola/Desktop/NFL 

### Final Preprocessing Steps

In [20]:
from Preprocessing_makers import preprocess_ep

In [21]:
ep_scale = preprocess_ep(extra_point_kick_df)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Clustering

In [22]:
import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
model = hdbscan.HDBSCAN()

In [ ]:
ep_scale = pd.DataFrame(ep_scale).dropna

In [ ]:
model_results = model.fit(ep_scale)

In [ ]:
model_results.labels_.max()

In [ ]:
ep_scale['cluster_id'] = model_results.labels_

In [ ]:
ep_scale['cluster_id'].value_counts()

In [ ]:
plt.figure(figsize = (16,18))
model.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())
plt.show()

In [ ]:
cluster0 = final_df[final_df['cluster_id']==0]
#cluster1 = final_df[final_df['cluster_id']==1]
#cluster2 = final_df[final_df['cluster_id']==2]

outliers = final_df[final_df['cluster_id']==-1]

In [ ]:
cluster0['specialTeamsResult'].value_counts()

In [ ]:
outliers['specialTeamsResult'].value_counts()